<a href="https://colab.research.google.com/github/harsha-bsm/LDA-on-REDMI-NOTE8-reviews-in-FLIPKART/blob/master/Preprocessing_and_Modelling_LDA_SAMSUNGM30.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import numpy as np
import pandas as pd

In [ ]:
data=pd.read_csv("samsungdata.csv")

In [ ]:
data.head()

,Unnamed: 0,Model,rating,user,time_of_review,review,title,user_address,user_type
0,0,Samsung Galaxy M30s Blue 128 Gb Reviews: Lates...,5,Aryan Patil,2months ago,Bokeh mode is very excellent than any other ha...,Wonderful,Latur District,Certified Buyer
1,1,Samsung Galaxy M30s Blue 128 Gb Reviews: Lates...,5,Flipkart Customer,5months ago,Just awsome product the picture quality is goo...,Great product,North Twenty Four Parganas District,Certified Buyer
2,2,Samsung Galaxy M30s Blue 128 Gb Reviews: Lates...,4,Suresh Kumar,5months ago,Samsung galaxy m30s is most useful.But Samsung...,Nice product,New Delhi,Certified Buyer
3,3,Samsung Galaxy M30s Blue 128 Gb Reviews: Lates...,1,Rohit Purba,6months ago,Not worth for me. i buy for my samsung tv this...,Utterly Disappointed,Rajpura,Certified Buyer
4,4,Samsung Galaxy M30s Blue 128 Gb Reviews: Lates...,5,Flipkart Customer,5months ago,"BSOLUTELY BEST PHONE, GO FOR THIS PHONELET ME ...",Highly recommended,Muzaffarpur,Certified Buyer


In [ ]:
pip install emot 

In [ ]:
####### Apply emoticons and emojis

from emot.emo_unicode import UNICODE_EMO, EMOTICONS # contains english text of emojis and emoticons in dict format 
def convert_emojis(review):
  for x in review:
    if x in UNICODE_EMO.keys():
      review=review.replace(x,"")
  return review

In [ ]:
#converting all basic contracted foms  
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [ ]:
#removing special characters and tabs ,spaces,new lines etc..
def removecharacters(review):
  review= re.sub('[^A-Za-z0-9]+',' ',review)  #anything exept numbers and alphabets, replace them with space
  review=re.sub(r"\n"," ",review)  #new lines into space
  review=re.sub(r"\t"," ",review)  #tabs into space
  review=re.sub(r"\v"," ",review)  #vertical tab into space
  review=re.sub(r"\s"," ",review)   #all extra spaces into single space
  return review  


In [ ]:
# https://gist.github.com/sebleier/554280
# we are removing the words from the stop words list: 'no', 'nor', 'not'
# <br /><br /> ==> after the above steps, we are getting "br br"
# we are including them into stop words list
# instead of <br /> if we have <br/> these tags would have revmoved in the 1st step
# ignore words like won't, haven't etc.. We have already taken care of them

stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

In [ ]:
data["review"]=data["review"].apply(lambda x:convert_emojis(x))
data["review"]=data["review"].apply(lambda x:x.lower()) 
data["review"]=data["review"].apply(lambda x:decontracted(x))
data["review"]=data["review"].apply(lambda x:removecharacters(x))

In [ ]:
data["review"]

0      bokeh mode is very excellent than any other ha...
1      just awsome product the picture quality is goo...
2      samsung galaxy m30s is most useful but samsung...
3      not worth for me i buy for my samsung tv this ...
4      bsolutely best phone go for this phonelet me c...
                             ...                        
293                                                 okay
294               super style and good look nice product
295                                                 good
296                                                 good
297                                         good product
Name: review, Length: 298, dtype: object

## Title Preprocessing

In [ ]:
data["title"]=data["title"].apply(lambda x:convert_emojis(x))
data["title"]=data["title"].apply(lambda x:x.lower()) 
data["title"]=data["title"].apply(lambda x:decontracted(x))
data["title"]=data["title"].apply(lambda x:removecharacters(x))

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models import ldamodel
import gensim.corpora
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
#https://scikit-learn.org/stable/auto_examples/applications/plot_topics_extraction_with_nmf_lda.html#sphx-glr-auto-examples-applications-plot-topics-extraction-with-nmf-lda-py
def print_top_words(model, feature_names, n_top_words):
  topics_cluster=[]
  for topic_idx, topic in enumerate(model.components_):
      message = "Topic #%d: " % topic_idx
      message += ",".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]])
      topics_cluster.append([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]])
      print(message)
  

### BOW and LDA 

In [ ]:
bow=CountVectorizer(stop_words=stopwords,ngram_range=(1,2))
bow_review=bow.fit_transform(data["review"])
df_bow=pd.DataFrame(bow_review.toarray(),columns=bow.get_feature_names())
lda = LatentDirichletAllocation(n_components=4,random_state=0)
lda.fit_transform(df_bow)
print_top_words(lda,bow.get_feature_names(),10)

Topic #0: super,battery,phone,camera,display,good,best,superb,mobile,quality
Topic #1: nice,phone,product,best,mobile,nice product,nice phone,nice mobile,not,best phone
Topic #2: good,phone,product,mobile,battery,camera,good product,not,performance,not good
Topic #3: not,product,samsung,camera,phone,bad,wesome,good,quality,buy


### TFIDF AND LDA

In [ ]:
tfidf=TfidfVectorizer(ngram_range=(1,1))
tfidf_words=tfidf.fit_transform(data["review"])

In [ ]:
words=tfidf.get_feature_names()


In [ ]:

df_bow=pd.DataFrame(tfidf_words.toarray(),columns=words)
lda = LatentDirichletAllocation(n_components=5,random_state=0)
lda.fit_transform(tfidf_words)
print_top_words(lda,words,10)

Topic #0: best,mobile,phone,and,battery,very,price,to,smartphone,hangs
Topic #1: mobile,superb,is,not,backup,the,ok,battery,price,phone
Topic #2: wesome,xcellent,money,awesome,product,for,value,flipkart,of,phone
Topic #3: good,very,product,nice,is,so,battery,samsung,and,no
Topic #4: nice,phone,super,it,is,camera,the,not,and,in


### NMF - TFIDF

In [ ]:
from sklearn.decomposition import NMF, LatentDirichletAllocation

In [ ]:
nmf = NMF(n_components=5, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(tfidf_words)

In [ ]:
print_top_words(nmf,words,10)

Topic #0: good,battery,not,is,one,backup,and,camera,so,normal
Topic #1: nice,phone,super,osm,camera,amazing,products,all,display,cell
Topic #2: product,wesome,xcellent,bad,money,no,smartphone,not,of,value
Topic #3: is,it,phone,best,and,the,this,camera,quality,in
Topic #4: very,mobile,superb,best,from,poor,samsung,phones,worst,m30s


### NMF - BOW

In [ ]:
nmf = NMF(n_components=4, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(bow_review)

In [ ]:
bow_words=bow.get_feature_names()

In [ ]:
print_top_words(nmf,bow_words,4)

Topic #0: good,camera,battery,nice
Topic #1: phone,best,super,nice phone
Topic #2: not,phone,bad,money
Topic #3: product,nice,mobile,nice product
